In [7]:
# ONLY RUN THIS ONCE. Uncomment right command for your OS before running.
# !source .venv/bin/activate (MacOS / Linux)
# !.venv\Scripts\activate (Windows)
!pip install streamlit pandas ollama plotly pdfplumber python-docx openpyxl

# tempfile? pathlib?
# difusedxml to defend against quadratic blowups/billion laughs attacks?

In [3]:
# WARNING run streamlit run app.py instead (otherwise shell scripts will be executed). Use ipynb for testing and development.
import streamlit as st
import pandas as pd
import ollama
import plotly.express as px
# Document processing
import pdfplumber
import docx
import sqlite3
import logging
llm = ollama.Client()
def process_with_llm(text):
    system_prompt = """You are a financial data analyst. Your tasks:
    1. Extract transaction data from bank statement documents (e.g., txt, pdf, csv, xml, word) common columns are: date, details, outflow, inflow, balance.
    2. Categorize transactions into hierarchical groups:
        - essentials: {groceries, utilities, rent}
        - transport: {fuel, public_transport, car_maintenance}
        - leisure: {dining, entertainment, shopping}
    3. Very concisely explain your categorization logic
    4. Return data in JSON format:
        {
            "transactions": [...],
            "categories": {...},
            "reasoning": "..."
        }
    """
    
    try:
        # system is overall behaviour, user is specific messages.
        response = llm.chat(model="deepseek-coder:1.3b", messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Process this bank statement:\n{text}"}
        ])
        return response["message"]["content"]
    except Exception as e:
        return f"Error processing text: {str(e)}"
    
def read_file(uploaded_file):
    # Warning: Not typesafe
    if uploaded_file.type == "application/pdf":
        with pdfplumber.open(uploaded_file) as pdf:
            return " ".join(page.extract_text() for page in pdf.pages)
    elif uploaded_file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
        doc = docx.Document(uploaded_file)
        return " ".join(paragraph.text for paragraph in doc.paragraphs)
    else:
        return uploaded_file.getvalue().decode()
    
def main():
    st.title("now.tech")
    uploaded_file = st.file_uploader("Upload document (txt, csv, docx, pdf)", 
                                   type=["txt", "csv", "docx", "pdf"],
                                   accept_multiple_files=False)
    if uploaded_file:
        text = read_file(uploaded_file)
        # get filename to determine table name
        filename = uploaded_file.name
        filename = filename.split('.')[0]
        tablename = None
        import re
        month_year_pattern = re.compile(r'[a-zA-Z]{3,4} \d{2,4}')
        find_pattern = filename.search(month_year_pattern)
        if find_pattern:
            tablename = find_pattern.group(0)
        else:
            tablename = filename
            

        with st.spinner("Processing..."):
            analysis = process_with_llm(text)
            st.write('### LLM Analysis')
            st.write(analysis)

        # save to local database
        model = 'deepseek-coder:1.3b'
        conn = sqlite3.connect('processed_data.db')
        df = pd.DataFrame({
            'original_text': [text],
            f'{model}_output': [analysis],
            'filename': [uploaded_file.name]
        })
        df.to_sql(tablename, conn, if_exists='append', index=False)
        st.success("Data saved to database")

        # display data
        st.write("### Processed Data")
        st.dataframe(pd.read_sql_query(f"SELECT * FROM {tablename}", conn))

        # Export to excel (hope API downloads, and configures to 2 sheets)
        df.to_excel("processed_data.xlsx", index=False)

        # Visualise data
        # fig_inflow = px.pie(df, values='inflow', names='category', title='Spending by Category')
        # fig_outflow = px.pie(df, values='outflow', names='category', title='Spending by Category')
        # st.plotly_chart(fig_inflow)
        # st.plotly_chart(fig_outflow)

        # Append additional data
        st.write("### Append additional data")
        additional_data = st.file_uploader("Upload additional data", type=["csv", "xlsx", "txt", "docx", "pdf"])
        if additional_data:
            logging.warning('We\'re not ready to append data yet')

if __name__ == "__main__":
    main()
